# Test deployed web application

This notebook uses some duplicate questions and tests them against the deployed web application on AKS.

In [1]:
import numpy as np
import pandas as pd
import requests
import json

from utilities import text_to_json

/anaconda/envs/MLAKSDeployment/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Get the external url for the web application running on AKS cluster.

In [3]:
service_json = !sudo kubectl get service azure-ml -o json
service_dict = json.loads(''.join(service_json))
app_url = service_dict['status']['loadBalancer']['ingress'][0]['ip']
app_url

'13.67.110.95'

Quickly check if the web application is working.

In [4]:
scoring_url = 'http://{}/score'.format(app_url)
version_url = 'http://{}/version'.format(app_url)
health_url = 'http://{}/'.format(app_url)

In [5]:
scoring_url

'http://13.67.110.95/score'

In [6]:
!curl $health_url

Healthy

In [7]:
!curl $version_url # Reports the lightgbm version

2.1.2

Let's use one of the duplicate questions to test our web service.

In [8]:
dupes_test_path = 'dupes_test.tsv'
dupes_test = pd.read_csv(dupes_test_path, sep='\t', encoding='latin1')
text_to_score = dupes_test.iloc[0,4]
text_to_score

"getting the length of a 'named' array?. i'm not sure what they are called, but what i mean is this:  length should be 2 here how can i see how many attributes i have in the array? array.length doesn't work =( i've been trying all kinds of things and i feel like i'm missing something really simple here.. thank you for your help"

In [9]:
jsontext = text_to_json(text_to_score)
jsontext[:100]

'{"input": "getting the length of a \'named\' array?. i\'m not sure what they are called, but what i mea'

In [14]:
headers = {'content-type': 'application/json'}
r = requests.post(scoring_url, data=jsontext, headers=headers) # Run the request twice since the first time takes a 
%time r = requests.post(scoring_url, data=jsontext, headers=headers) # little longer due to the loading of the model
print(r)
r.json()

CPU times: user 1.99 ms, sys: 573 µs, total: 2.57 ms
Wall time: 290 ms
<Response [200]>


{'result': [[[5223, 6700, 0.9056762423642606],
   [126100, 4889658, 0.5897041951019686],
   [7364150, 7364307, 0.013095757762535671],
   [19590865, 19590901, 0.009726747791608314],
   [11922383, 11922384, 0.007400362725375734],
   [5627284, 5627301, 0.006518820196008976],
   [1771786, 1771824, 0.004990243605901752],
   [24900875, 24900924, 0.004120260321857448],
   [7837456, 14853974, 0.0016726596738734054],
   [1129216, 1129270, 0.0014618991064883388],
   [237104, 1473742, 0.0009983357055414952],
   [784929, 784946, 0.000918666929043429],
   [23667086, 23667087, 0.0007934756119722436],
   [4616202, 4616273, 0.0006798101809810278],
   [750486, 750506, 0.0005087117457724936],
   [14028959, 8716680, 0.0004299633021580329],
   [14220321, 14220323, 0.00037730639500855277],
   [85992, 86014, 0.00030718180657696116],
   [695050, 695053, 0.00029608261688253765],
   [1822350, 1822769, 0.00024789271341276657],
   [10693845, 10693852, 0.00022361713661220404],
   [5767325, 5767357, 0.000194494936

Let's try a few more duplicate questions and display their top 3 original matches.

In [15]:
dupes_to_score = dupes_test.iloc[:5,4]

In [16]:
results = [requests.post(scoring_url, data=text_to_json(text), headers=headers) for text in dupes_to_score]

Let's print top 3 matches for each duplicate question.

In [17]:
[results[i].json()['result'][0][0:3] for i in range(0, len(results))]

[[[5223, 6700, 0.9056762423642606],
  [126100, 4889658, 0.5897041951019686],
  [7364150, 7364307, 0.013095757762535671]],
 [[750486, 750506, 0.2690727074211682],
  [5223, 6700, 0.03662320274754268],
  [14220321, 14220323, 0.01004067672291757]],
 [[27928, 27943, 0.9997417254204117],
  [14220321, 14220323, 0.06305885438195412],
  [23667086, 23667087, 0.02452617277905266]],
 [[27928, 27943, 0.9998416785304494],
  [23667086, 23667087, 0.11296631038987077],
  [14220321, 14220323, 0.07512705718114801]],
 [[31044, 31047, 0.9830364526746855],
  [14220321, 14220323, 0.09503445047637052],
  [203198, 1207393, 0.0858315642150876]]]

Next let's quickly check what the request response performance is for the deployed model on AKS cluster.

In [18]:
text_data = list(map(text_to_json, dupes_to_score)) # Retrieve the text data

In [19]:
timer_results = list()
for text in text_data:
    res=%timeit -r 1 -o -q requests.post(scoring_url, data=text, headers=headers)
    timer_results.append(res.best)

In [20]:
timer_results

[0.17760504340039915,
 0.1358339660000638,
 0.19734794579999287,
 0.19250325269968016,
 0.175964030000614]

In [21]:
print('Average time taken: {0:4.2f} ms'.format(10**3 * np.mean(timer_results)))

Average time taken: 175.85 ms


We have tested that the model works and we can now move on to the [next notebook to get a sense of its throughput](07_Speed_Test_WebApp.ipynb).